In [1]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import numpy as np

UsageError: Line magic function `%` not found.


In [ ]:
peter_dice = [1, 2, 3, 4]
colin_dice = range(1, 7)

def p_pmf():
    l = []
    for i in itertools.product(range(1,5), repeat=9):
        l.append(sum(i))
    d = Counter(l)
    return {k: v / float(len(l)) for k, v in d.iteritems()}

def c_pmf():
    l = []
    for i in itertools.product(range(1,7), repeat=6):
        l.append(sum(i))
    d = Counter(l)
    return {k: v / float(len(l)) for k, v in d.iteritems()}

pe_pmf = (pd.DataFrame(p_pmf(), index=[1]).T)
cr_pmf = (pd.DataFrame(c_pmf(), index=[1]).T)

In [3]:
def convolute(X, Y):
    ## P(P > C) = P(P - C > 0)
    ## Z = X - Y  Y = Z + X (k)
    ## P(Z = z) = Sum_6^36 P(X = k) P(Y = z + k)
    Zmin = 9 - 36
    Zmax = 36 - 6
    
    Z = pd.DataFrame(columns = ["Prob"], index = range(Zmin, Zmax + 1))

    for z in range(Zmin, Zmax + 1):
        Zs = 0
        for k in range(6, 37):
            try:
                Zs += X[1].ix[k] * Y[1].ix[z + k]
            except:
                Zs += 0
        Z.Prob.ix[z] = Zs
    return Z

round(1 - sum(convolute(pe_pmf, cr_pmf).ix[0:30]["Prob"]), 7)

0.5731441

In [6]:
## P[X > Y] = P[X > Y | Y = 6] + ... + P[X > Y | Y = 36] etc etc
## => P[X > Y] = Sum^36_k=6 P[X > k]P[Y = k]

def prob_x_gt_y():
    total = 0
    for i in range(6, 36):
        if i < 9:
            total += 1 * cr_pmf.ix[i][1]
        else:
            total += sum(pe_pmf.ix[i + 1:37][1]) * cr_pmf.ix[i][1]
    return total


round(prob_x_gt_y(), 7)

0.5731441